# Case loans

In [1]:
# Setup

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


# Data

## Import

In [2]:
# link to data
ROOT = "https://raw.githubusercontent.com/kirenz/modern-statistics/main/data/"
DATA = "loans.csv"
 
df = pd.read_csv(ROOT + DATA)

## Overview

In [3]:
df

emp_title  emp_length state homeownership  annual_income  \
0     global config engineer          3.0    NJ      mortgage        90000.0   
1      warehouse office clerk        10.0    HI          rent        40000.0   
2                    assembly         3.0    WI          rent        40000.0   
3            customer service         1.0    PA          rent        30000.0   
4        security supervisor         10.0    CA          rent        35000.0   
...                       ...         ...   ...           ...            ...   
9995                   owner         10.0    TX          rent       108000.0   
9996                 director         8.0    PA      mortgage       121000.0   
9997                toolmaker        10.0    CT      mortgage        67000.0   
9998                  manager         1.0    WI      mortgage        80000.0   
9999       operations analyst         3.0    CT          rent        66000.0   

      verified_income  debt_to_income  annual_income_joint  \
0            Verified           18.01                  NaN   
1        Not Verified            5.04                  NaN   
2     Source Verified           21.15                  NaN   
3        Not Verified           10.16                  NaN   
4            Verified           57.96              57000.0   
...               ...             ...                  ...   
9995  Source Verified           22.28                  NaN   
9996         Verified           32.38                  NaN   
9997         Verified           45.26             107000.0   
9998  Source Verified           11.99                  NaN   
9999     Not Verified           20.82                  NaN   

     verification_income_joint  debt_to_income_joint  ...  sub_grade  \
0                          NaN                   NaN  ...         C3   
1                          NaN                   NaN  ...         C1   
2                          NaN                   NaN  ...         D1   
3                          NaN                   NaN  ...         A3   
4                     Verified                 37.66  ...         C3   
...                        ...                   ...  ...        ...   
9995                       NaN                   NaN  ...         A4   
9996                       NaN                   NaN  ...         D3   
9997           Source Verified                 29.57  ...         E2   
9998                       NaN                   NaN  ...         A1   
9999                       NaN                   NaN  ...         B4   

      issue_month  loan_status  initial_listing_status  disbursement_method  \
0        Mar-2018      Current                   whole                 Cash   
1        Feb-2018      Current                   whole                 Cash   
2        Feb-2018      Current              fractional                 Cash   
3        Jan-2018      Current                   whole                 Cash   
4        Mar-2018      Current                   whole                 Cash   
...           ...          ...                     ...                  ...   
9995     Jan-2018      Current                   whole                 Cash   
9996     Feb-2018      Current                   whole                 Cash   
9997     Feb-2018      Current              fractional                 Cash   
9998     Feb-2018      Current                   whole                 Cash   
9999     Feb-2018      Current                   whole                 Cash   

       balance  paid_total  paid_principal  paid_interest  paid_late_fees  
0     27015.86     1999.33          984.14        1015.19             0.0  
1      4651.37      499.12          348.63         150.49             0.0  
2      1824.63      281.80          175.37         106.43             0.0  
3     18853.26     3312.89         2746.74         566.15             0.0  
4     21430.15     2324.65         1569.85         754.80             0.0  
...        ...         ...             ...  

In [4]:
print(f"We have {len(df.index):,} observations and {len(df.columns)} columns in our dataset.")

We have 10,000 observations and 55 columns in our dataset.


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 55 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   emp_title                         9167 non-null   object 
 1   emp_length                        9183 non-null   float64
 2   state                             10000 non-null  object 
 3   homeownership                     10000 non-null  object 
 4   annual_income                     10000 non-null  float64
 5   verified_income                   10000 non-null  object 
 6   debt_to_income                    9976 non-null   float64
 7   annual_income_joint               1495 non-null   float64
 8   verification_income_joint         1455 non-null   object 
 9   debt_to_income_joint              1495 non-null   float64
 10  delinq_2y                         10000 non-null  int64  
 11  months_since_last_delinq          4342 non-null   float64
 12  earli

## Data Corrections

In [6]:
df['state'] = df['state'].astype('category')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 55 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   emp_title                         9167 non-null   object  
 1   emp_length                        9183 non-null   float64 
 2   state                             10000 non-null  category
 3   homeownership                     10000 non-null  object  
 4   annual_income                     10000 non-null  float64 
 5   verified_income                   10000 non-null  object  
 6   debt_to_income                    9976 non-null   float64 
 7   annual_income_joint               1495 non-null   float64 
 8   verification_income_joint         1455 non-null   object  
 9   debt_to_income_joint              1495 non-null   float64 
 10  delinq_2y                         10000 non-null  int64   
 11  months_since_last_delinq          4342 non-null   float

In [7]:
# Convert to categorical

cat_convert = ['emp_title', 'homeownership', 'verified_income']

for i in cat_convert:
    df[i] = df[i].astype("category")

In [8]:
df["metric"] = df["annual_income"] / df["total_credit_limit"]

## Variable lists

In [9]:
# list of all numerical data
list_num = df.select_dtypes(include=[np.number]).columns.tolist()


In [10]:
# list of all categorical data
list_cat = df.select_dtypes(include=['category']).columns.tolist()
list_cat

['emp_title', 'state', 'homeownership', 'verified_income']

## Preparation for data splitting

In [11]:
# define outcome variable as y_label
y_label = 'interest_rate'

# select features
features = df.drop(columns=[y_label]).columns.tolist()

# create feature data for data splitting
X = df[features]

# list of numeric features
feat_num = X.select_dtypes(include=[np.number]).columns.tolist()

# list of categorical features
feat_cat = X.select_dtypes(include=['category']).columns.tolist() 

# create response for data splitting
y = df[y_label]

# Train Test split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Exploration Data

In [15]:
df_train = pd.DataFrame(X_train.copy())
df_train = df_train.join(pd.DataFrame(y_train))

df_train

emp_title  emp_length state homeownership  \
9254             software developper         3.0    WA          rent   
1561                             NaN         NaN    TX      mortgage   
1670                              rn         2.0    GA      mortgage   
6087                  director of it         1.0    GA      mortgage   
6669         assistant section chief        10.0    MD      mortgage   
...                              ...         ...   ...           ...   
5734        field service technician        10.0    NC      mortgage   
5191   director, quality improvement         9.0    GA      mortgage   
5390                     food server         7.0    NV          rent   
860                  anesthesia tech         5.0    PA      mortgage   
7270  technical training coordinator        10.0    GA          rent   

      annual_income  verified_income  debt_to_income  annual_income_joint  \
9254       112000.0         Verified           11.20                  NaN   
1561        30000.0  Source Verified           16.52                  NaN   
1670        80000.0  Source Verified           17.99                  NaN   
6087       140000.0  Source Verified            3.44                  NaN   
6669        83500.0  Source Verified           30.06                  NaN   
...             ...              ...             ...                  ...   
5734        59000.0  Source Verified           11.33                  NaN   
5191       235000.0  Source Verified            8.83                  NaN   
5390        32000.0  Source Verified            8.70                  NaN   
860         44000.0     Not Verified           20.61                  NaN   
7270        87000.0  Source Verified            7.37                  NaN   

     verification_income_joint  debt_to_income_joint  ...  loan_status  \
9254                       NaN                   NaN  ...      Current   
1561                       NaN                   NaN  ...      Current   
1670                       NaN                   NaN  ...      Current   
6087                       NaN                   NaN  ...      Current   
6669                       NaN                   NaN  ...   Fully Paid   
...                        ...                   ...  ...          ...   
5734                       NaN                   NaN  ...   Fully Paid   
5191                       NaN                   NaN  ...      Current   
5390                       NaN                   NaN  ...      Current   
860                        NaN                   NaN  ...      Current   
7270                       NaN                   NaN  ...      Current   

      initial_listing_status  disbursement_method   balance    paid_total  \
9254                   whole                 Cash  32559.72   3493.670000   
1561                   whole                 Cash  13645.89   2106.490000   
1670              fractional                 Cash  22987.54   4368.170000   
6087                   whole                 Cash  10623.12    831.750000   
6669                   whole            DirectPay      0.00  11237.191111   
...                      ...                  ...       ...           ...   
5734                   whole                 Cash      0.00   8170.514047   
5191              fractional                 Cash  35192.45   6527.140000   
5390                   whole                 Cash  11588.87    847.190000   
860                    whole                 Cash  11688.12    495.770000   
7270                   whole                 Cash  18550.27   1878.700000   

      paid_principal  paid_interest  paid_late_fees    metric  interest_rate  
9254         2440.28        1053.39             0.0  1.898305          12.61  
1561         1354.11         752.38             0.0  0.616650          15.05  
1670         3012.46        1355.71             0.0  0.413603          13.59  
6087          376.88         454.87             0.0  0.514814          15.04  
6669        11200.00        